In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


In [16]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

In [3]:
batch_size = 32
num_classes = 10
epochs = 200
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [4]:
np.random.seed(4)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_complete =np.concatenate((x_train,x_test),axis=0)
y_complete =np.concatenate((y_train,y_test),axis=0)
rand_train = np.random.randint(0,49999,size=10000)
rand_test= np.random.randint(50000,59999,size=2000)
x_train = x_complete[rand_train]
y_train = y_complete[rand_train]
x_test = x_complete[rand_test]
y_test=y_complete[rand_test]
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (10000, 32, 32, 3)
10000 train samples
2000 test samples


In [5]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [6]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

In [7]:
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

In [8]:
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [9]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [10]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Using real-time data augmentation.
Epoch 1/200
312/312 [==============================] - 10s - loss: 2.1001 - acc: 0.2197 - val_loss: 1.9200 - val_acc: 0.3255
Epoch 2/200
312/312 [==============================] - 6s - loss: 1.8368 - acc: 0.3339 - val_loss: 1.7576 - val_acc: 0.3715
Epoch 3/200
312/312 [==============================] - 6s - loss: 1.7380 - acc: 0.3686 - val_loss: 1.7458 - val_acc: 0.3590
Epoch 4/200
312/312 [==============================] - 6s - loss: 1.6698 - acc: 0.3921 - val_loss: 1.6101 - val_acc: 0.4275
Epoch 5/200
312/312 [==============================] - 6s - loss: 1.6211 - acc: 0.4004 - val_loss: 1.5771 - val_acc: 0.4270
Epoch 6/200
312/312 [==============================] - 6s - loss: 1.5676 - acc: 0.4270 - val_loss: 1.5822 - val_acc: 0.4305
Epoch 7/200
312/312 [==============================] - 6s - loss: 1.5419 - acc: 0.4400 - val_loss: 1.4516 - val_acc: 0.4820
Epoch 8/200
312/312 [==============================] - 6s - loss: 1.4858 - acc: 0.4646 - val_los

312/312 [==============================] - 6s - loss: 0.7521 - acc: 0.7403 - val_loss: 0.8207 - val_acc: 0.7195
Epoch 67/200
312/312 [==============================] - 6s - loss: 0.7472 - acc: 0.7419 - val_loss: 0.8721 - val_acc: 0.7035
Epoch 68/200
312/312 [==============================] - 6s - loss: 0.7411 - acc: 0.7419 - val_loss: 0.8252 - val_acc: 0.7225
Epoch 69/200
312/312 [==============================] - 6s - loss: 0.7254 - acc: 0.7457 - val_loss: 0.8500 - val_acc: 0.7200
Epoch 70/200
312/312 [==============================] - 6s - loss: 0.7399 - acc: 0.7428 - val_loss: 0.8186 - val_acc: 0.7130
Epoch 71/200
312/312 [==============================] - 6s - loss: 0.7203 - acc: 0.7494 - val_loss: 0.8997 - val_acc: 0.7035
Epoch 72/200
312/312 [==============================] - 6s - loss: 0.7222 - acc: 0.7467 - val_loss: 0.7995 - val_acc: 0.7375
Epoch 73/200
312/312 [==============================] - 6s - loss: 0.7342 - acc: 0.7460 - val_loss: 0.8362 - val_acc: 0.7165
Epoch 74/200


312/312 [==============================] - 6s - loss: 0.5439 - acc: 0.8180 - val_loss: 0.7932 - val_acc: 0.7545
Epoch 132/200
312/312 [==============================] - 6s - loss: 0.5257 - acc: 0.8184 - val_loss: 0.7547 - val_acc: 0.7530
Epoch 133/200
312/312 [==============================] - 6s - loss: 0.5317 - acc: 0.8199 - val_loss: 0.7137 - val_acc: 0.7745
Epoch 134/200
312/312 [==============================] - 6s - loss: 0.5255 - acc: 0.8158 - val_loss: 0.7921 - val_acc: 0.7570
Epoch 135/200
312/312 [==============================] - 6s - loss: 0.5423 - acc: 0.8140 - val_loss: 0.7302 - val_acc: 0.7570
Epoch 136/200
312/312 [==============================] - 6s - loss: 0.5195 - acc: 0.8209 - val_loss: 0.7502 - val_acc: 0.7560
Epoch 137/200
312/312 [==============================] - 6s - loss: 0.5336 - acc: 0.8157 - val_loss: 0.7413 - val_acc: 0.7650
Epoch 138/200
312/312 [==============================] - 6s - loss: 0.5374 - acc: 0.8167 - val_loss: 0.7950 - val_acc: 0.7470
Epoch 

312/312 [==============================] - 6s - loss: 0.4833 - acc: 0.8327 - val_loss: 0.7381 - val_acc: 0.7700
Epoch 197/200
312/312 [==============================] - 6s - loss: 0.4754 - acc: 0.8400 - val_loss: 0.7334 - val_acc: 0.7685
Epoch 198/200
312/312 [==============================] - 6s - loss: 0.4665 - acc: 0.8416 - val_loss: 0.7789 - val_acc: 0.7670
Epoch 199/200
312/312 [==============================] - 6s - loss: 0.4721 - acc: 0.8404 - val_loss: 0.7620 - val_acc: 0.7615
Epoch 200/200
312/312 [==============================] - 6s - loss: 0.4758 - acc: 0.8405 - val_loss: 0.7721 - val_acc: 0.7620


In [11]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Load label names to use in prediction results
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'

Saved trained model at C:\Users\mites\Documents\Cognitive\saved_models\keras_cifar10_trained_model.h5 


In [12]:
keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

In [13]:

# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

Model Accuracy = 0.74


In [17]:
fig = plt.figure(figsize=(8,3))
for i in range(num_classes):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(y_train[:]==i)[0]
    features_idx = x_train[idx,::]
    img_num = np.random.randint(features_idx.shape[0])
    im = np.transpose(features_idx[img_num,::],(1,2,0))
    #ax.set_title(class_names[i])
    plt.imshow(im)
plt.show()

TypeError: Invalid dimensions for image data

In [14]:
for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Actual Label = dog vs. Predicted Label = horse
Actual Label = frog vs. Predicted Label = frog
Actual Label = horse vs. Predicted Label = horse
Actual Label = truck vs. Predicted Label = truck
Actual Label = horse vs. Predicted Label = horse
Actual Label = deer vs. Predicted Label = horse
Actual Label = bird vs. Predicted Label = bird
Actual Label = cat vs. Predicted Label = airplane
Actual Label = deer vs. Predicted Label = bird
Actual Label = truck vs. Predicted Label = truck
Actual Label = automobile vs. Predicted Label = automobile
Actual Label = dog vs. Predicted Label = cat
Actual Label = ship vs. Predicted Label = ship
Actual Label = dog vs. Predicted Label = bird
Actual Label = dog vs. Predicted Label = bird
Actual Label = bird vs. Predicted Label = bird
Actual Label = frog vs. Predicted Label = frog
Actual Label = cat vs. Predicted Label = deer
Actual Label = cat vs. Predicted Label = horse
Actual Label = automobile vs. Predicted Label = automobile
Actual Label = deer vs. Predi